In [25]:
from define import qcode as qc
import numpy as np
from define.QECCLfid import uncorrectable as uc
from define.QECCLfid import minwt as mw
from define import globalvars as gv
from define import chanreps as chrep
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [31]:
def ChangeOrdering(probs, old_order, new_order):
    """
    Change from an ordering in [L,S,T] to another.
    This uses numpy reshape and transpose.
    old = L S T
    new = T L S
    perm = 2 0 1
    """
    t_size = np.power(2, qcode.N - qcode.K, dtype=np.int)
    s_size = t_size
    l_size = np.power(4, qcode.K, dtype=np.int)
    sizes = {"T": t_size, "S": s_size, "L": l_size}
    probs_reshaped = np.reshape(
        probs, [sizes[old_order[0]], sizes[old_order[1]], sizes[old_order[2]]]
    )
    perm = [old_order.index(ax) for ax in new_order]
    probs_reordered = np.reshape(np.transpose(probs_reshaped, perm), -1)
    return probs_reordered

In [28]:
qcode = qc.QuantumErrorCorrectingCode("Steane")
qc.Load(qcode)
qc.populate_symplectic(qcode)
mw.ComputeCorrectableIndices(qcode)
#correctable_set = np.array(qcode.Paulis_correctable, dtype=np.int)
sqprobs = np.random.rand(4)
sqprobs[0] = 1-0.66**10
sqprobs[1:] = (1-sqprobs[0]) * sqprobs[1:]/np.sum(sqprobs[1:])
print(sqprobs[0])
probs = np.tile(sqprobs, [qcode.N, 1])

0.9843166311908921


In [29]:
pauli_probs = np.ones(qcode.PauliOperatorsLST.shape[0], dtype=np.double)
for i in range(qcode.PauliOperatorsLST.shape[0]):
    for q in range(qcode.N):
        pauli_probs[i] *= probs[q, qcode.PauliOperatorsLST[i, q]]
print(pauli_probs)

[8.95248829e-01 1.86844514e-06 1.86844514e-06 ... 3.49685857e-11
 3.49685857e-11 2.65798751e-09]


In [ ]:
mw.ComputeUncorrProbs(np.tile(probs, [qcode.N, 1]), qcode, 3)

In [ ]:
%lprun -f mw.ComputeUncorrProbs mw.ComputeUncorrProbs(np.tile(probs, [qcode.N, 1]), qcode, 3)

In [66]:
uncorrs = np.load("/Users/pavi/Documents/chbank/29_05_2020_13_05_35/results/phy_uncorr.npy")
infids = np.load("/Users/pavi/Documents/chbank/29_05_2020_13_05_35/results/log_infid.npy")

In [67]:
infids[:, 1]

array([2.06481638e-05], dtype=float128)

In [68]:
uncorrs[:,1]

array([2.06481638e-05])

In [17]:
np.max(np.abs(uncorrs[:, 1] - infids[:, 1]))

0.026931575886987316193

In [32]:
pvec = ChangeOrdering(pauli_probs, "LST", "TLS")

In [33]:
pvec

array([8.95248829e-01, 3.78116405e-12, 3.78116405e-12, ...,
       3.44335943e-11, 3.44335943e-11, 2.65798751e-09])

In [37]:
pauli_probs.shape

(16384,)

In [38]:
pvec.shape

(16384,)

In [55]:
nstabs = 2**6
nlogs = 4
l, s, t = np.random.randint(0,4,size=(1,)), np.random.randint(0,64,size=(1,)), np.random.randint(0,64,size=(1,))
tls_order = t * nstabs * nlogs + l * nstabs + s
lst_order = l * nstabs * nstabs + s * nstabs + t
print("l = {}, s = {}, t = {}\ntls_order = {}\nlst_order = {}".format(l, s, t, tls_order, lst_order))
print("pauli_probs[lst_order] = {}\npvec[tls_order] = {}.".format(pauli_probs[lst_order], pvec[tls_order]))

l = [2], s = [24], t = [48]
tls_order = [12440]
lst_order = [9776]
pauli_probs[lst_order] = [4.53009689e-13]
pvec[tls_order] = [4.53009689e-13].


In [72]:
(48 * 15)

720